In [23]:
%pip install qiskit qiskit-ibm-runtime
%pip install qiskit
%pip install qiskit[visualization]
%pip install pylatexenc
%pip install qiskit qiskit-aer --upgrade


In [36]:
from qiskit.circuit.library import PhaseOracle
from qiskit import QuantumCircuit

#expression = "(x1 & x2 & x3)"
expression = "(~x1 & x2 & x3)"
oracle = PhaseOracle(expression)

qc = QuantumCircuit(3)
qc = qc.compose(oracle)

print(oracle)
print(qc.draw())

        
q_0: ─o─
      │ 
q_1: ─■─
      │ 
q_2: ─■─
        
        
q_0: ─o─
      │ 
q_1: ─■─
      │ 
q_2: ─■─
        


/tmp/ipython-input-3378544040.py:6: DeprecationWarning: The class ``qiskit.circuit.library.phase_oracle.PhaseOracle`` is deprecated as of Qiskit 2.2. It will be removed in Qiskit 3.0. Use the class qiskit.circuit.library.PhaseOracleGate instead.
  oracle = PhaseOracle(expression)


In [25]:
def diffuser(n):
    qc = QuantumCircuit(n)

    # Apply transformation |s> -> |00..0> (H-gates)
    qc.h(range(n))

    # Apply transformation |00..0> -> |11..1> (X-gates)
    qc.x(range(n))

    # Do multi-controlled-Z gate
    qc.h(n-1)
    qc.mct(list(range(n-1)), n-1)  # multi-controlled-toffoli
    qc.h(n-1)

    # Apply transformation |11..1> -> |00..0>
    for qubit in range(n):
        qc.x(qubit)

    # Apply transformation |00..0> -> |s>
    qc.h(range(n))

    # We will return the diffuser as a gate
    gate = qc.to_gate()
    gate.name = "diffuser"

    return gate



In [26]:
from qiskit import *
from qiskit.visualization import *
def grover(n):
    # The circuit is a Grover's search for the all-ones state.
    var = QuantumRegister(n, 'var')
    out = QuantumRegister(1, 'out')
    cr = ClassicalRegister(n, 'c')
    qc = QuantumCircuit(var, out, cr)

    # Initialize the output qubit to a phase-flip.
    qc.x(n)
    qc.h(n)

    # Apply the Hadamard gate to every qubit.
    qc.h(var)
    qc.barrier()

    # Apply the oracle to every qubit.
    #qc.append(oracle(n), range(n+1))
    qc.compose(oracle, inplace=True)
    qc.barrier()

    # Apply the diffuser to every qubit.
    qc.append(diffuser(n), range(n))
    qc.barrier()

    # Undo the output qubit phase-flip.
    #qc.h(n)
    #qc.x(n)

    qc.measure(var, cr)

    return qc

In [27]:
from qiskit_aer import AerSimulator
def execute(qc):
    shots = 128
    result = AerSimulator().run(qc, shots=shots, memory=True).result()
    return result


In [28]:
qc = grover(3)
qc.draw()


AttributeError: 'QuantumCircuit' object has no attribute 'mct'

Nuova implementazione


In [38]:
from qiskit_aer import AerSimulator
from qiskit.circuit.library import PhaseOracle
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# Boolean expression d
#expr = "(x1 & x2 & x3)"   # |111>
#expr = "(~x1 & x2 & x3)"  # |110>
expr = "(~x1 & x2 & ~x3)"  # |010>


oracle = PhaseOracle(expr)


n = oracle.num_qubits


qc = QuantumCircuit(n, n)


qc.h(range(n))


qc.compose(oracle, inplace=True)

# diffuser
qc.h(range(n))
qc.x(range(n))
qc.h(n-1); qc.mcx(list(range(n-1)), n-1); qc.h(n-1)
qc.x(range(n))
qc.h(range(n))


qc.measure(range(n), range(n))

print(qc.draw())


result = AerSimulator().run(qc, shots=128, memory=True).result()
counts = result.get_counts()

print("\nRisultati:")
print(counts)
plot_histogram(counts)
plt.show()


     ┌───┐        ┌───┐┌───┐               ┌───┐┌───┐     ┌─┐      
q_0: ┤ H ├──────o─┤ H ├┤ X ├────────────■──┤ X ├┤ H ├─────┤M├──────
     ├───┤      │ ├───┤├───┤            │  ├───┤├───┤     └╥┘┌─┐   
q_1: ┤ H ├──────■─┤ H ├┤ X ├────────────■──┤ X ├┤ H ├──────╫─┤M├───
     ├───┤┌───┐ │ ├───┤├───┤┌───┐┌───┐┌─┴─┐├───┤├───┤┌───┐ ║ └╥┘┌─┐
q_2: ┤ H ├┤ X ├─■─┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├─╫──╫─┤M├
     └───┘└───┘   └───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘ ║  ║ └╥┘
c: 3/══════════════════════════════════════════════════════╩══╩══╩═
                                                           0  1  2 

Risultati:
{'001': 1, '110': 1, '100': 2, '111': 6, '011': 6, '101': 6, '000': 6, '010': 100}


/tmp/ipython-input-280577414.py:12: DeprecationWarning: The class ``qiskit.circuit.library.phase_oracle.PhaseOracle`` is deprecated as of Qiskit 2.2. It will be removed in Qiskit 3.0. Use the class qiskit.circuit.library.PhaseOracleGate instead.
  oracle = PhaseOracle(expr)
